In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from scipy.stats import linregress

In [2]:
random_state = 42
seeds = np.arange(100)
#seeds = [42]

In [3]:
data = pd.read_csv('..\\Data\\ze41_mol_desc_db_red.csv', header=0, sep=';', decimal=',')

In [4]:
col_names = data.columns
x_cols = ['P_VSA_MR_5', 'Mor04m', 'E1p', 'Mor22s', 'LUMO / eV']
#x_cols = ['VE2_G/D', 'Eig14_EA(dm)', 'Mor31m', 'TDB04u', 'HATS1e']
X = data[col_names[3:]]
y = data[col_names[1]]

In [5]:
#sz = len(x_cols)
sz = 63

In [6]:
def scale_x(X_train, X_valid):
    scalex = MinMaxScaler(feature_range=(-1,1))
    scalex.fit(X_train)
    return [pd.DataFrame(scalex.transform(x), columns=X.columns) for x in [X_train, X_valid]]

In [7]:
def scale_y(y_train, y_valid):
    scaley = MinMaxScaler(feature_range=(0, 1))
    scaley.fit(y_train)
    return [pd.DataFrame(scaley.transform(y), columns=y.columns) for y in [y_train, y_valid]] + [scaley]

In [8]:
def get_model():
    model = keras.models.Sequential([
        keras.layers.GaussianNoise(stddev=0.1, input_shape=(sz,)),
        keras.layers.Dense(50, activation='relu'),
        keras.layers.Dense(20, activation='relu'),
        keras.layers.Dense(10, activation='relu'),
        keras.layers.Dense(1)
        ])
    model.compile(
        optimizer=tf.optimizers.Adam(learning_rate=0.01),
        loss='mean_squared_error')
    return model

In [9]:
def rmse(x, y):
    return np.sqrt(((x-y)**2).mean())

In [10]:
def evaluate(predictions, y_valid):
    means = predictions[predictions.columns[1:]].mean(axis=1).to_numpy()
    yv = y_valid['inhibition efficiency ZE41 / %'].to_numpy()
    r,p = linregress(means, yv)[2:4]
    return [r**2, rmse(yv, means), r, p]

In [11]:
kf = KFold(n_splits=10, random_state=random_state, shuffle=True)

In [12]:
values_all = []
pred_0 = None

In [13]:
for train_index, test_index in kf.split(X):
    X_train = X.iloc[train_index, :]
    X_valid = X.iloc[test_index, :]
    y_train = pd.DataFrame(y.iloc[train_index])
    y_valid = pd.DataFrame(y.iloc[test_index])
    X_train_sc, X_valid_sc = scale_x(X_train, X_valid)
    y_train_sc, y_valid_sc, scaley = scale_y(y_train, y_valid)
    losses = []
    predictions = pd.DataFrame(y_valid)

    for seed in seeds:
        tf.keras.backend.clear_session()
        tf.random.set_seed(seed)
        idxs = np.random.default_rng(seed=seed).choice(len(col_names)-3, size=sz, replace=False)
        model = get_model()
        #X_train_sel = X_train_sc[['CATS2D_03_AP', 'CATS3D_03_AP', 'CATS3D_02_AP', 'P_VSA_MR_5', 'LUMO / eV']]
        #X_val_sel = X_valid_sc[['CATS2D_03_AP', 'CATS3D_03_AP', 'CATS3D_02_AP', 'P_VSA_MR_5', 'LUMO / eV']]
        X_train_sel = X_train_sc[X_train_sc.columns[idxs]]
        X_val_sel = X_valid_sc[X_valid.columns[idxs]]
        history = model.fit(X_train_sel, y_train_sc, validation_data=(X_val_sel, y_valid_sc), epochs=25, verbose=0)
        losses.append(history.history['val_loss'][-1])
        y_pred = model.predict(tf.convert_to_tensor(X_val_sel))
        predictions[seed] = scaley.inverse_transform(y_pred)
    
    if pred_0 is None:
        pred_0 = predictions.copy()
    values_all.append(evaluate(predictions, y_valid))

In [14]:
values_all = pd.DataFrame(values_all)

In [15]:
values_all.columns=(['R^2', 'RMSE', 'r', 'p'])

In [16]:
values_all

,R^2,RMSE,r,p
0,0.576696,49.986510,0.759405,0.079865
1,0.006460,100.289846,0.080377,0.879695
2,0.844439,89.843642,0.918934,0.009591
3,0.374447,71.481425,0.611920,0.196685
4,0.490828,57.484135,0.700591,0.121048
5,0.023780,143.638582,0.154209,0.770520
6,0.495351,49.974412,0.703812,0.118599
7,0.783095,28.783058,0.884926,0.019101
8,0.457532,54.974112,0.676411,0.140123
9,0.728682,51.718027,0.853628,0.030569


In [17]:
values_all['RMSE'].to_numpy()

array([ 49.9865103 , 100.2898459 ,  89.84364155,  71.48142522,
        57.48413491, 143.63858232,  49.97441226,  28.7830579 ,
        54.97411166,  51.71802714])

In [18]:
values_all[values_all.index != 5].median(axis=0)

R^2      0.495351
RMSE    54.974112
r        0.703812
p        0.118599
dtype: float64

In [19]:
values_all.median(axis=0)

R^2      0.493089
RMSE    56.229123
r        0.702201
p        0.119824
dtype: float64

In [20]:
values_all.mean(axis=0)

R^2      0.478131
RMSE    69.817375
r        0.634421
p        0.236580
dtype: float64

In [21]:
values_all.std(axis=0)

R^2      0.286786
RMSE    33.254577
r        0.289905
p        0.316697
dtype: float64

In [22]:
for i in range(10):
    v = values_all.iloc[i, :]
    print('{} & {:.2f} & {:.0f} & {:.2f} & {:.2f} \\\\'.format(i, v[0], v[1], v[2], v[3]))

0 & 0.58 & 50 & 0.76 & 0.08 \\
1 & 0.01 & 100 & 0.08 & 0.88 \\
2 & 0.84 & 90 & 0.92 & 0.01 \\
3 & 0.37 & 71 & 0.61 & 0.20 \\
4 & 0.49 & 57 & 0.70 & 0.12 \\
5 & 0.02 & 144 & 0.15 & 0.77 \\
6 & 0.50 & 50 & 0.70 & 0.12 \\
7 & 0.78 & 29 & 0.88 & 0.02 \\
8 & 0.46 & 55 & 0.68 & 0.14 \\
9 & 0.73 & 52 & 0.85 & 0.03 \\


In [23]:
col_names[1]

'inhibition efficiency ZE41 / %'

In [24]:
i = 0
for train_index, test_index in kf.split(X):
    print(i, test_index)
    i += 1

0 [ 0  5 13 36 45 54]
1 [12 33 46 48 50 57]
2 [ 3  6  8 17 31 52]
3 [ 4 19 34 40 43 58]
4 [ 9 15 25 27 30 56]
5 [11 16 24 26 32 55]
6 [ 1 29 37 41 44 53]
7 [ 2 21 23 35 39 47]
8 [10 18 20 22 49 59]
9 [ 7 14 28 38 42 51]


In [25]:
data.iloc[[11, 16, 24, 26, 32, 55], :]

,compound,inhibition efficiency ZE41 / %,LinIE ZE41,MW,AMW,Mv,Mi,nTA,RBF,nDB,...,CATS3D_06_NL,CATS3D_03_LL,CATS3D_04_LL,CATS3D_05_LL,HOMO / eV,LUMO / eV,Hlgap / eV,Cv / kJ/(mol-K)@293.15K,Cp / kJ/(mol-K)@293.15K,chem_pot / kJ/mol@293.15K
11,34-Dihydroxybenzoicacid,-270,0.000,154.13,9.066,0.673,1.123,4.0,0.059,1.0,...,0.0,0.0,0.0,0.0,-5.808,-1.476,4.332,0.151608,0.159923,232.78
16,asparagine,-188,0.238,132.14,7.773,0.575,1.169,5.0,0.188,2.0,...,0.0,0.0,0.0,0.0,-5.912,-0.361,5.551,0.146506,0.154820,258.03
24,cysteine,-104,0.481,121.18,8.656,0.587,1.149,4.0,0.154,1.0,...,0.0,0.0,0.0,0.0,-5.942,-0.647,5.295,0.120800,0.129114,193.95
26,diglycolicacid,60,0.957,134.10,8.940,0.610,1.153,4.0,0.286,2.0,...,0.0,0.0,0.0,0.0,-7.064,-0.457,6.607,0.133403,0.141717,187.93
32,glycine,-215,0.159,75.08,7.508,0.550,1.175,3.0,0.111,1.0,...,0.0,0.0,0.0,0.0,-6.188,-0.311,5.877,0.078541,0.086856,131.06
55,salicylicacid,37,0.890,138.13,8.633,0.670,1.117,3.0,0.063,1.0,...,0.0,0.0,0.0,0.0,-6.027,-1.909,4.117,0.126638,0.134952,229.62


In [26]:
data[data['compound'] == '4-hydroxybenzoicacid']

,compound,inhibition efficiency ZE41 / %,LinIE ZE41,MW,AMW,Mv,Mi,nTA,RBF,nDB,...,CATS3D_06_NL,CATS3D_03_LL,CATS3D_04_LL,CATS3D_05_LL,HOMO / eV,LUMO / eV,Hlgap / eV,Cv / kJ/(mol-K)@293.15K,Cp / kJ/(mol-K)@293.15K,chem_pot / kJ/mol@293.15K
4,4-hydroxybenzoicacid,-170,0.29,138.13,8.633,0.67,1.117,3.0,0.063,1.0,...,0.0,0.0,0.0,0.0,-6.198,-1.472,4.726,0.132298,0.140612,225.54


In [27]:
from scipy.stats import pearsonr

In [28]:
y_p = predictions[predictions.columns[1:]].mean(axis=1)

In [29]:
pearsonr(y_p.to_numpy(), y_valid['inhibition efficiency ZE41 / %'].to_numpy())

(0.8536284595963719, 0.03056896387042919)

In [30]:
tr_f = np.array([-157, 39, 38, 12, -6, -17])
yp_f = pred_0[pred_0.columns[1:]].mean(axis=1).to_numpy()

In [31]:
r_f, p_f = linregress(tr_f, yp_f)[2:4]
print('{:.2f}, {:.0f}, {:.2f}, {:.3f}'.format(r_f**2, rmse(tr_f, yp_f), r_f, p_f))

0.58, 50, 0.76, 0.080


In [32]:
tr_x = np.delete(tr_f, 2)
yp_x = np.delete(yp_f, 2)

In [33]:
r_x, p_x = linregress(tr_x, yp_x)[2:4]
print('{:.2f}, {:.0f}, {:.2f}, {:.3f}'.format(r_x**2, rmse(tr_x, yp_x), r_x, p_x))

0.94, 38, 0.97, 0.007


In [34]:
values_all.mean(axis=0)

R^2      0.478131
RMSE    69.817375
r        0.634421
p        0.236580
dtype: float64

In [35]:
pred_0[pred_0.columns[1:]].mean(axis=1)

0     -76.119690
5      17.626371
13    125.407600
36     10.130041
45    -23.077417
54     -9.369327
dtype: float32

In [36]:
pred_0[pred_0.columns[1:]].std(axis=1)

0     53.268280
5     39.898960
13    93.965012
36    50.454472
45    42.412323
54    45.811993
dtype: float32